In [1]:
%run Main.ipynb

# Redshift Drift 

## - Spectroscopic velocity

$\Delta v = \frac{c \Delta z}{1+z} = c H_0 \Delta t \left[1 - \frac{E(z)}{1+z}\right] = kh \left[1 - \frac{E(z)}{1+z}\right]$, where $H_0 = 100 H_{100}$ and $k = c H_{100}\delta t$

Assuming $\delta t = 1$ year we have $k=3.064$ cm/s.

In [2]:
def spect_velocity(zs, h, OmegaM, w0, wa, NrYears):
    if w0 == None: 
        w0 = -1.
    if wa == None: 
        wa = 0
    return c100t1 * NrYears * h * ( 1 - 1/(1+zs) * function_E_z(zs, OmegaM, w0, wa))

## - Spectroscopic velocity uncertainty

#### Liske Approach - (Approximation of the spectroscopic velocity uncertainty for ELT)
$\sigma_{\Delta v} = 1.35 \frac{2370}{S/N} \sqrt{\frac{30}{N_{QSO}})} \left({\frac{5}{1+z_{QSO}}}\right)^x$, with $x=1.7$ for $z\leqslant4$ and $x=0.9$ for $z>4$ 

In [3]:
def uncLiske_spect_velocity(zs):
    NrQSO = 6
    SN    = 3000
    unc_zELT = np.zeros_like(zs)
    for i in np.arange(len(zs)):
        if zs[i] <= 4:
            x = 1.7
        else:
            x = .9
        unc_zELT[i] = 1.35 * 2370/SN * np.sqrt(30/NrQSO) * (5/(1+zs[i]))**x
    return unc_zELT

## - Derivatives calculation

In [4]:
def dfdh_z(z, h, OmegaM, w0, wa, NrYears):
    return ( spect_velocity(z, h+hh, OmegaM, w0, wa, NrYears)-spect_velocity(z, h, OmegaM, w0, wa, NrYears) )/hh

def dfdOmegaM_z(z, h, OmegaM, w0, wa, NrYears):
    return ( spect_velocity(z, h,OmegaM+hh, w0, wa, NrYears)-spect_velocity(z, h, OmegaM, w0, wa, NrYears) )/hh

def dfdw0_z(z, h, OmegaM, w0, wa, NrYears):
    if w0 != None:
        return ( spect_velocity(z, h, OmegaM, w0+hh, wa, NrYears)-spect_velocity(z, h, OmegaM, w0, wa, NrYears) )/hh
    else:
        return 0

def dfdwa_z(z, h, OmegaM, w0, wa, NrYears):
    if wa != None:
        return ( spect_velocity(z, h,OmegaM, w0, wa+hh, NrYears)-spect_velocity(z, h, OmegaM, w0, wa, NrYears) )/hh
    else:
        return 0

# Measurements
### - Redshifts

In [5]:
zLiske = np.array([2.,  2.5, 3.,  3.5, 4.5])

### - Uncertainties

In [6]:
unc_zLiske = uncLiske_spect_velocity(zLiske)

# Priors

In [7]:
unch = 0.05
uncOmegaM = 0.03
uncw0 = 0.1
uncwa = 0.4

# Fisher Matrices
## - No Priors

#### CPL

In [8]:
CPLT_Liske_np = FisherMatrix_z(zLiske, unc_zLiske, h, OmegaM, w0=-0.8, wa=-0.2, NrYears=20, analysis = 'redshift')
CPLF_Liske_np = FisherMatrix_z(zLiske, unc_zLiske, h, OmegaM, w0=-0.9, wa=0.3, NrYears=20, analysis = 'redshift')

## - With Priors

#### CPL

In [9]:
#Priors in h,OmegaM,w0,wa 
CPLT_Liske = add1DPriors(CPLT_Liske_np, [[unch,0],[uncOmegaM,1],[uncw0,2], [uncwa,3]])
CPLF_Liske = add1DPriors(CPLF_Liske_np, [[unch,0],[uncOmegaM,1],[uncw0,2], [uncwa,3]])